## Using different featurizations with different dimensions
GB1_2016 has combintorially measured all the possible mutations and been studied extensively in the previous literature. Here, we use GB1_2016 as an example to explore many different possible settings in ODBO and collect the corresponding results. We will walk through this notebook with detailed comments.
To use this notebook, please change the global control varibles and read the comments.

In [ ]:
import numpy as np
import torch
import pandas as pd
import odbo
import os

### Global control varibles

In [ ]:
# Experiment settings 
dataset_name ='GB1_2016'
random_seed = 0 #Random seed for the trial
search_iter = 50 #Number of new observations, GB1_2014=100, BRCA1=50, avGFP_2016=50
encoding = 'georgiev'
if encoding == 'onehot':
    index_data = np.load('example_protein_onehot_IndexToCombo.npy')
    feature_data = np.load('example_protein_onehot_UnNormalized.npy').reshape(160000,80)
elif encoding == 'georgiev':
    index_data = np.load('example_protein_georgiev_IndexToCombo.npy')
    feature_data = np.load('example_protein_georgiev_Normalized.npy').reshape(160000,76)
    
# BO method settings (Optimization step)
BO_method = 'TuRBO' #Must be 'BO' or 'TuRBO'
gp_method='gp_regression' #Must be 'gp_regression' or 'robust_regression'
acqfn = 'ei'
tr_length = [6.4] #Trust region length, used in the TuRBO. 
batch_size = 1 #Number of new oberservations provided by BO. We found 1 is the most cost-effective experimentally
failure_tolerance =10 #Number of failure iterations to change TR length in TuRBO
save_files = True #Save files or not
np.random.seed(random_seed)


### Data initalization

In [ ]:
# Load dataset
data_test = pd.read_csv('../datasets/GB1_2016_149361.csv', sep=',')
name, Y_test = np.array(data_test['AACombo']), np.array(data_test['Fitness'])
#Load the preselected indices using a certain shuffling order. Control Round 0 experiments to be the same for different trials
if os.path.isfile('sele_experiment_GB1_2016.npy') == True:
    name_sele = np.load('sele_experiment_GB1_2016.npy')
    Y_train = np.load('sele_fitness_GB1_2016.npy')
else:
    # Let each site has 20 AA codes at least show up twice 
    sele_indices = odbo.initialization.initial_design(name, least_occurance=2*np.ones(name.shape[1]),allow_abundance=allow_abundance, update_method=update_method,verbose = True)
    # Initial experiments are selected to be name_sele with fitness of Y_sele
    name_sele, Y_train = name[sele_indices], Y_test[sele_indices]
print('Selected initial experiments no. is ', len(Y_train))
print('Select max Y: ', Y_train.max())


In [ ]:
test_indice = []
fullname = []
for i in range(40):
    fullname.append(''.join(name_sele[i, :]))
fullname = np.array(fullname)
for i in range(name.shape[0]):
    a = np.where(name[i] == fullname)[0]
    if len(a) == 0 :
        test_indice.append(i)
name = name[test_indice]
Y_test = Y_test[test_indice]

### Featurization and find the adaptive search space model

In [ ]:
### Featurization and find the adaptive search space model
X_train, X_test = [],[]
for i in range(name_sele.shape[0]):
    a = np.where(''.join(name_sele[i, :])==index_data)[0][0]
    X_train.append(feature_data[a])
X_train = np.vstack(X_train)
for i in range(name.shape[0]):
    a = np.where(''.join(name[i])==index_data)[0][0]
    X_test.append(feature_data[a])
X_test = np.vstack(X_test)
print(X_train.shape, X_test.shape)
np.save('X_test.npy', X_test)
np.save('X_train.npy', X_train)

### Run optimization

In [ ]:
# Creat search data
# shuffle_order = np.arange(len(Y_test))
np.random.shuffle(shuffle_order)
X_test = X_test[shuffle_order]
Y_test = Y_test[shuffle_order]
X_train_sele, Y_train_sele = torch.tensor(X_train), torch.tensor(Y_train.reshape(len(Y_train),1))
X_test_sele, Y_test_sele = torch.tensor(X_test), torch.tensor(Y_test.reshape(len(Y_test),1))
print(X_train_sele.shape, X_test_sele.shape)
## Run BO experiment with robust regression or directly gp
l, failure_count = 0, 0 #l is the current iteration, failure_count controls when to switch to another featurization
if BO_method == 'TuRBO':
    state = odbo.turbo.TurboState(dim=X_train_sele.shape[1], batch_size=batch_size, length=tr_length, n_trust_regions=len(tr_length), failure_tolerance = failure_tolerance)
    state.best_value = Y_train_sele.max()

while l < search_iter:
    print("Iter: ", l, "Current Max: ", Y_train_sele.max().detach().numpy(), "Test max: ", Y_test_sele.max().detach().numpy())
    active_space = []
    for j in range(X_train_sele.shape[1]):
        a=np.where(X_train_sele[0, j] == X_train_sele[1:, j])[0]
        if len(a) != len(X_train_sele)-1:
            active_space.append(j)
    print(len(active_space))
    X_train_sele_active = X_train_sele[:, active_space]
        
    # Run optimization using different methods    
    if BO_method == 'BO':
        X_next, acq_value, next_exp_id = odbo.bo_design(X=X_train_sele_active, Y=Y_train_sele, X_pending=X_test_sele, gp_method=gp_method, batch_size=batch_size, acqfn=acqfn)
    elif BO_method == 'TuRBO':
        X_next, acq_value, raw_next_exp_id = odbo.turbo_design(state=state, X=X_train_sele_active, Y=Y_train_sele, X_pending=X_test_sele, n_trust_regions=len(tr_length), batch_size=batch_size, gp_method=gp_method, acqfn=acqfn)
        Y_next_m = torch.zeros((len(tr_length), batch_size, 1), device=Y_train_sele.device, dtype=Y_train_sele.dtype)
        next_exp_id = []  
        for i in range(batch_size):
            next_exp_id_m = raw_next_exp_id[:, i]
            Y_next_m[:, i, 0], idtoadd = Y_test_sele[next_exp_id_m].reshape(len(tr_length)), next_exp_id_m[np.argmax(Y_test_sele[next_exp_id_m])]
            next_exp_id.append(idtoadd)
    # Update the newly find experimental value to the current training set, and remove that point from test set
    Y_train_sele = torch.cat([Y_train_sele, Y_test_sele[next_exp_id]])
    ids_keep = list(np.delete(range(Y_test_sele.shape[0]), next_exp_id))
    Y_test_sele = Y_test_sele[ids_keep]
    X_train_sele = torch.cat((X_train_sele, X_test_sele[next_exp_id]))
    X_test_sele = X_test_sele[ids_keep]
    print("Newly added value: ", Y_train_sele[-batch_size:].detach().numpy())
    l = l+1

# Save the BO results. Note we save all the observations including the Round 0 ones
if save_files:
    if acqfn == 'ei':
        if gp_method == 'robust_regression':
            np.save('results/{}/{}_{}_RobustGP_batch{}_{}_{}.npy'.format(dataset_name, dataset_name, BO_method, batch_size, encoding, random_seed), Y_train_sele)
        elif gp_method == 'gp_regression':
            np.save('results/{}/{}_{}_GP_batch{}_{}_{}.npy'.format(dataset_name, dataset_name, BO_method, batch_size, encoding, random_seed), Y_train_sele)
    else:
        if gp_method == 'robust_regression':
            np.save('results/{}/{}_{}_RobustGP_batch{}_{}_{}_{}.npy'.format(dataset_name, dataset_name, BO_method, batch_size, encoding, acqfn, random_seed), Y_train_sele)
        elif gp_method == 'gp_regression':
            np.save('results/{}/{}_{}_GP_batch{}_{}_{}_{}.npy'.format(dataset_name, dataset_name, BO_method, batch_size, encoding, acqfn, random_seed), Y_train_sele)

        